In [1]:
import pandas as pd
#df=pd.read_csv('https://cloud.minsa.gob.pe/s/To2QtqoNjKqobfw/download', usecols=['FECHA_CORTE', 'EDAD', 'SEXO', 'FECHA_VACUNACION', 'DOSIS', 'DEPARTAMENTO', 'FABRICANTE'],parse_dates=['FECHA_VACUNACION'])
df=pd.read_csv('vacunas_covid.csv', usecols=['FECHA_CORTE', 'EDAD', 'SEXO', 'FECHA_VACUNACION', 'DOSIS', 'DEPARTAMENTO', 'FABRICANTE'], parse_dates=['FECHA_VACUNACION'])
fecha_corte=df['FECHA_CORTE'].drop_duplicates().set_axis(['fecha_corte'])
#fecha_corte.to_json("../resultados/fecha_corte.json")

In [2]:
#DIARIO DOSIS 1 Y DOSIS 2
df_ambas_dosis=df[['FECHA_VACUNACION','DOSIS','SEXO']].groupby(['FECHA_VACUNACION','DOSIS']).count()
df_ambas_dosis=df_ambas_dosis.reset_index()
df_ambas_dosis=df_ambas_dosis.pivot(index='FECHA_VACUNACION', columns='DOSIS', values='SEXO')
df_ambas_dosis=df_ambas_dosis.rename_axis(None, axis=1)
df_ambas_dosis.columns=['DOSIS1','DOSIS2']
df_ambas_dosis=df_ambas_dosis.fillna(0).astype('int')
df_ambas_dosis

,DOSIS1,DOSIS2
FECHA_VACUNACION,,
2021-02-08,1,0
2021-02-09,3749,0
2021-02-10,16580,1
2021-02-11,34099,0
2021-02-12,32280,0
...,...,...
2021-09-12,69831,42099
2021-09-13,31789,11189
2021-09-14,35378,10343


In [3]:
#ACUMULADO DOSIS 1 Y DOSIS 2
df_ambas_dosis_cum=df_ambas_dosis.cumsum()
df_ambas_dosis_cum

,DOSIS1,DOSIS2
FECHA_VACUNACION,,
2021-02-08,1,0
2021-02-09,3750,0
2021-02-10,20330,1
2021-02-11,54429,1
2021-02-12,86709,1
...,...,...
2021-09-12,12301798,8902249
2021-09-13,12333587,8913438
2021-09-14,12368965,8923781


In [4]:
col_poblacion=[426806,
1180638,
430736,
1497438,
668213,
1453711,
1129854,
1357075,
365317,
760267,
975182,
1361467,
2016771,
1310785,
10628470,
1027559,
173811,
192740,
271904,
2047954,
1237997,
899648,
370974,
251521,
589110]


In [5]:
# ACUMULADO POR DEPARTAMENTO (DOSIS 2 - VACUNACION COMPLETA)
df_ambas_dosis_departamento=df[['DEPARTAMENTO','DOSIS','SEXO']].groupby(['DEPARTAMENTO', 'DOSIS']).count()
df_ambas_dosis_departamento=df_ambas_dosis_departamento.reset_index()
df_ambas_dosis_departamento=df_ambas_dosis_departamento.pivot(index='DEPARTAMENTO', columns='DOSIS', values='SEXO')
df_ambas_dosis_departamento.columns=['DOSIS1','DOSIS2']
df_ambas_dosis_departamento['POBLACION']=col_poblacion
df_ambas_dosis_departamento['INDICE']=round(df_ambas_dosis_departamento['DOSIS2']/(df_ambas_dosis_departamento['POBLACION']/100000)).astype('int')
df_ambas_dosis_departamento

,DOSIS1,DOSIS2,POBLACION,INDICE
DEPARTAMENTO,,,,
AMAZONAS,140660,87880,426806,20590
ANCASH,464180,335318,1180638,28401
APURIMAC,173651,109434,430736,25406
AREQUIPA,631545,461815,1497438,30840
AYACUCHO,213724,132262,668213,19793
CAJAMARCA,479551,319358,1453711,21968
CALLAO,579318,435640,1129854,38557
CUSCO,444355,334015,1357075,24613
HUANCAVELICA,127767,94706,365317,25924


In [6]:
# ACUMULADO POR GRUPO ETARIO (DOSIS 2 - VACUNACION COMPLETA)
bins = [18,30,40,50,60,70,80,df['EDAD'].max()+1]
labels = ['18 a 29 años','30 a 39 años','40 a 49 años','50 a 59 años','60 a 69 años','70 a 79 años','80 años a más']
poblacion_por_grupo_etario = [6303670,5031117,4183174,3277134,2221241,1271842,647355] 
df_edades = df
df_edades['GRUPO_ETARIO'] = pd.cut(df['EDAD'], bins=bins, labels=labels, right=False)
df_edades = df_edades[df_edades.DOSIS == 2].groupby(['GRUPO_ETARIO'])["DOSIS"].count().reset_index()
df_edades.rename(columns = {'DOSIS':'DOSIS2'}, inplace = True)
df_edades['POBLACION']=poblacion_por_grupo_etario
df_edades['PORCENTAJE']=round(df_edades['DOSIS2']/df_edades['POBLACION']*100,2)
df_edades=df_edades.set_index('GRUPO_ETARIO')
df_edades

,DOSIS2,POBLACION,PORCENTAJE
GRUPO_ETARIO,,,
18 a 29 años,407665,6303670,6.47
30 a 39 años,688107,5031117,13.68
40 a 49 años,2195346,4183174,52.48
50 a 59 años,2311140,3277134,70.52
60 a 69 años,1761703,2221241,79.31
70 a 79 años,1036426,1271842,81.49
80 años a más,536001,647355,82.80


In [7]:
#DIARIO POR FABRICANTE
df_fabricante=df[['FECHA_VACUNACION','FABRICANTE','SEXO']].groupby(['FECHA_VACUNACION','FABRICANTE']).count()
df_fabricante=df_fabricante.reset_index()
df_fabricante=df_fabricante.pivot(index='FECHA_VACUNACION', columns='FABRICANTE', values='SEXO')
df_fabricante=df_fabricante.rename_axis(None, axis=1)
df_fabricante=df_fabricante.fillna(0).astype('int')
df_fabricante

,ASTRAZENECA,PFIZER,SINOPHARM
FECHA_VACUNACION,,,
2021-02-08,0,1,0
2021-02-09,0,0,3749
2021-02-10,0,0,16581
2021-02-11,0,1,34098
2021-02-12,1,2,32277
...,...,...,...
2021-09-12,264,48607,63059
2021-09-13,498,15618,26862
2021-09-14,183,18656,26882


In [8]:
#ACUMULADO POR FABRICANTE
df_fabricante_cum=df_fabricante.cumsum()
df_fabricante_cum

,ASTRAZENECA,PFIZER,SINOPHARM
FECHA_VACUNACION,,,
2021-02-08,0,1,0
2021-02-09,0,1,3749
2021-02-10,0,1,20330
2021-02-11,0,2,54428
2021-02-12,1,4,86705
...,...,...,...
2021-09-12,889779,13608419,6705849
2021-09-13,890277,13624037,6732711
2021-09-14,890460,13642693,6759593


In [9]:
from requests_html import HTMLSession
session = HTMLSession()
r = session.get('https://www.gob.pe/pongoelhombro#contador-de-vacunados')
scrap_contador_vacunados = r.html.find('.font-bold.text-3xl')
d = []
for number in scrap_contador_vacunados:
    d.append(number.text)
contador_vacunados = pd.Series(d,index=['total_dosis1', 'total_dosis2','ayer_dosis1','ayer_dosis2','total_dosis','ayer_total_dosis','vacunacion_completa'])
#contador_vacunados.to_json('../resultados/pongo_el_hombro.json')

In [10]:
#DIARIO DOSIS 1 Y DOSIS 2: TACNA
df_tacna=df[df['DEPARTAMENTO'] == 'TACNA']
df_diario_tacna=df_tacna[['FECHA_VACUNACION','DOSIS','SEXO']].groupby(['FECHA_VACUNACION','DOSIS']).count()
df_diario_tacna=df_diario_tacna.reset_index()
df_diario_tacna=df_diario_tacna.pivot(index='FECHA_VACUNACION', columns='DOSIS', values='SEXO')
df_diario_tacna=df_diario_tacna.rename_axis(None, axis=1)
df_diario_tacna.columns=['DOSIS1','DOSIS2']
df_diario_tacna=df_diario_tacna.fillna(0).astype('int')
df_diario_tacna

,DOSIS1,DOSIS2
FECHA_VACUNACION,,
2021-02-10,2,0
2021-02-11,724,0
2021-02-12,446,0
2021-02-13,165,0
2021-02-15,33,0
...,...,...
2021-09-10,1299,219
2021-09-11,1764,149
2021-09-13,293,78


In [12]:
#TOTAL DE 11 A 16 AÑOS
bins = [11,12,13,14,15,16,17]
labels = ['11 años','12 años','13 años','14 años','15 años','16 años'] 
df_11_16 = df
df_11_16['GRUPO_ETARIO'] = pd.cut(df['EDAD'], bins=bins, labels=labels, right=False)
df_11_16 = df_11_16[df_11_16.DOSIS == 2].groupby(['GRUPO_ETARIO'])["DOSIS"].count().reset_index()
df_11_16.rename(columns = {'DOSIS':'DOSIS2'}, inplace = True)
df_11_16=df_11_16.set_index('GRUPO_ETARIO')
df_11_16

,DOSIS2
GRUPO_ETARIO,
11 años,69
12 años,591
13 años,638
14 años,697
15 años,730
16 años,803


In [15]:
#ACUMULADO DE 11 A 16 AÑOS
df_11_16_diario = df
df_11_16_diario = df[df['EDAD'].isin([11,12,13,14,15,16])]
df_11_16_diario = df_11_16_diario[df_11_16_diario.DOSIS == 2].groupby(['FECHA_VACUNACION'])["DOSIS"].count().reset_index()
df_11_16_diario = df_11_16_diario.set_index('FECHA_VACUNACION')
df_11_16_diario.rename(columns = {'DOSIS':'DOSIS2'}, inplace = True)
df_11_16_diario_cum = df_11_16_diario.cumsum()
df_11_16_diario_cum

,DOSIS2
FECHA_VACUNACION,
2021-04-29,1
2021-05-27,2
2021-06-14,3
2021-07-07,4
2021-07-09,16
...,...
2021-09-12,3291
2021-09-13,3391
2021-09-14,3454
